In [1]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM
import transformers
import torch

print(transformers.__version__)

model_id = 'decapoda-research/llama-7b-hf'

nf4_config = BitsAndBytesConfig(
 load_in_4bit=True,
 bnb_4bit_quant_type='nf4',
 bnb_4bit_use_double_quant=True,
 bnb_4bit_compute_dtype=torch.float16
)

model_nf4 = AutoModelForCausalLM.from_pretrained(
  model_id,
  quantization_config=nf4_config,
  trust_remote_code=True,
)

4.31.0


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [2]:
print(model_nf4.get_memory_footprint())

3796381696


In [3]:
print(model_nf4)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
    

In [4]:
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_id)

pipeline = pipeline(
  'text-generation',
  model=model_nf4,
  tokenizer=tokenizer,
  torch_dtype=torch.float16,
  trust_remote_code=True,
  device_map='auto',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [5]:
prompt = """
A dialog, where User interacts with AI. AI is helpful, kind, obedient, honest, and knows its own limits. 
User: Hello, AI.
AI: Hello! How can I assist you today?
"""

output = pipeline(
  prompt,
  max_length=200,
  do_sample=True,
  top_k=10,
  num_return_sequences=1,
  eos_token_id=tokenizer.eos_token_id,
)

print(output[0]['generated_text'])

/home/kiddos/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(



A dialog, where User interacts with AI. AI is helpful, kind, obedient, honest, and knows its own limits. 
User: Hello, AI.
AI: Hello! How can I assist you today?
User: How much does this new dress cost?
AI: 23.63
User: Thanks. I think I'm going to buy a dress, a purse, and a hat.
AI: You seem to have made up your mind, but you may need some help deciding. 1. Which is more important to you: style or comfort?
2. How do you feel about your appearance?
User: I'm very vain but I also think clothes should be practical.
User: I think it's important for girls to have good-quality clothes, but I also care about being comfortable.
AI: I understand. 1. Which do you prefer?
